In [ ]:
# Import necessary modules
import pandas as pd 
import numpy as np
from numpy import *
import geocoder
import folium
from folium import plugins
from folium.plugins import HeatMap
from geopy.geocoders import GoogleV3
import requests
import sklearn
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

# Read the data from the file and store it in a dataframe
cbr_geo = pd.read_csv('indir/cbr-neighbourhood-geo.txt', sep = "\t")

# Inspect the first few rows of the dataframe
print(cbr_geo.head())

# Check the dimensions of the dataframe
print("The dataframe has {} rows and {} columns".format(cbr_geo.shape[0], cbr_geo.shape[1]))

In [ ]:
# Initialize lists to store latitude and longitude
Lat_list = []
Lng_list = []

# Loop through the rows of the dataframe
for i in tqdm(range(cbr_geo.shape[0])):
    # Geocode the address and store the latitude and longitude in the corresponding lists
    address = '{}, Canberra, Australia'.format(cbr_geo.at[i, 'Neighborhood'])
    g = geocoder.arcgis(address)
    Lat_list.append(g.latlng[0])
    Lng_list.append(g.latlng[1])

# Add the latitude and longitude columns to the dataframe
cbr_geo['Latitude'] = Lat_list
cbr_geo['Longitude'] = Lng_list

# Check the dimensions of the dataframe
print("The dataframe has {} rows and {} columns".format(cbr_geo.shape[0], cbr_geo.shape[1]))

# Inspect the first few rows of the dataframe
print(cbr_geo.head())

from geopy.geocoders import Nominatim

# create geolocator object
geolocator = Nominatim(user_agent="canberra_explorer")

# get latitude and longitude for the given address
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create a base map of Canberra using latitude and longitude values
map_cbr = folium.Map(location=[latitude, longitude], zoom_start=11, tiles="CartoDB dark_matter")

# add markers to map
for i in range(len(cbr_geo)):
    label = folium.Popup(cbr_geo.iloc[i]['Neighborhood'], parse_html=True)
    folium.CircleMarker(
        [cbr_geo.iloc[i]['Latitude'], cbr_geo.iloc[i]['Longitude']],
        radius=7.5,
        tooltip=label,
        popup=label,
        color='darkgreen',
        fill=True,
        fill_color='green',
        fill_opacity=0.8,
        parse_html=False).add_to(map_cbr)

map_cbr


In [ ]:
# initialise Foursquare credentials, version no., and limt

version = '20180604'
limit = 100

# create a function to lookup venues and iterate across in Canberra neighborhoods dataset

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            os.environ['FSQ_CLIENT_ID'], 
            os.environ['FSQ_CLIENT_SECRET'], 
            version, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # create the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
# run the above function on each neighborhood and create a new dataframe called Canberra venues

cbr_venues = getNearbyVenues(names=cbr_geo['Neighborhood'],
                                   latitudes=cbr_geo['Latitude'],
                                   longitudes=cbr_geo['Longitude']
                                  )